### ▶ 주식시장의 변수를 적용한 pygame ◀


In [ ]:
매매할종목의코드입력='sym'
import winsound
import time
import pyttsx3

sym=symbol="005930" 
symbol_name="삼성전자"
engine = pyttsx3.init()
engine.say(f"{symbol_name} .")
engine.runAndWait()

print(f"{symbol_name} ({sym})")


In [ ]:
hamsu="▶기본 함수를 실행합니다."
import requests
import json
import datetime
import time
import yaml
import random
import pyttsx3
import sys
from IPython.display import clear_output

# C:\Users\고창섭\Documents\GitHub\hantostock\863_풍차_kodex인버스2X\config_863.yaml
# C:\Users\고창섭\Documents\GitHub\hantostock\yaml폴더
###################################
ca_number=863
######################################################################
if ca_number == 863:
    with open(r"C:\KCS\Desktop\hantostock\yaml폴더\config_863.yaml", encoding='UTF8') as f:
        _cfg = yaml.load(f, Loader=yaml.FullLoader)
    f.close()

#########################################################################    
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

def send_message(msg):
    """디스코드 메세지 전송"""
    now = datetime.datetime.now()
    message = {"content": f"[{now.strftime('%Y-%m-%d %H:%M:%S')}] {str(msg)}"}
    requests.post(DISCORD_WEBHOOK_URL, data=message)
    print(message)

def get_access_token():
    """토큰 발급"""
    headers = {"content-type":"application/json"}
    body = {"grant_type":"client_credentials",
    "appkey":APP_KEY, 
    "appsecret":APP_SECRET}
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"
    res = requests.post(URL, headers=headers, data=json.dumps(body))
    ACCESS_TOKEN = res.json()["access_token"]
    return ACCESS_TOKEN
    
def hashkey(datas):
    """암호화"""
    PATH = "uapi/hashkey"
    URL = f"{URL_BASE}/{PATH}"
    headers = {
    'content-Type' : 'application/json',
    'appKey' : APP_KEY,
    'appSecret' : APP_SECRET,
    }
    res = requests.post(URL, headers=headers, data=json.dumps(datas))
    hashkey = res.json()["HASH"]
    return hashkey

def get_stock_balance():
    """주식 잔고조회"""
    PATH = "uapi/domestic-stock/v1/trading/inquire-balance"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8434R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "AFHR_FLPR_YN": "N",
        "OFL_YN": "",
        "INQR_DVSN": "02",
        "UNPR_DVSN": "01",
        "FUND_STTL_ICLD_YN": "N",
        "FNCG_AMT_AUTO_RDPT_YN": "N",
        "PRCS_DVSN": "01",
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": ""
    }
    res = requests.get(URL, headers=headers, params=params)
    try:
        stock_list = res.json()['output1']
        stock_avg = {}
        stock_qty ={}
        ord_psbl_qty = {}
        # send_message(f"====주식 보유잔고====")
        for stock in stock_list:
            if int(stock['hldg_qty']) > 0:
                stock_avg[stock['pdno']] = stock['pchs_avg_pric']   ##매입평균가격(pchs_avg_pric)
                stock_qty[stock['pdno']] = stock['hldg_qty']   ##보유수량
                ord_psbl_qty[stock['pdno']] = stock['ord_psbl_qty']  # 매도 가능 수량
        return stock_avg,stock_qty,ord_psbl_qty  ##stock_dic={'032350':'4500'}   즉 코드번호와 매입평균가격이 문자형 딕셔러니로 return된다.
    except:
        stock_avg['122630']=1 ; stock_qty['122630']=1 ; ord_psbl_qty['122630']=1
        return stock_avg,stock_qty,ord_psbl_qty
    
def get_balance():
    """미수없는 금액 조회"""
    PATH = "uapi/domestic-stock/v1/trading/inquire-psbl-order"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8908R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "PDNO": "114800",
        "ORD_UNPR": "65500",
        "ORD_DVSN": "01",
        "CMA_EVLU_AMT_ICLD_YN": "Y",
        "OVRS_ICLD_YN": "Y"
    }
    res = requests.get(URL, headers=headers, params=params)
    cash = res.json()['output']['nrcvb_buy_amt']
    return int(cash)

def get_current_price(code="114800"):
    """현재가 조회"""
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010100"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)
    try:
        current_price=int(res.json()['output']['stck_prpr'])
        
        return current_price
    except:
        current_price=100000
        return current_price

def get_priceImformation(code="005930"):
    """시작가, 최고가, 최저가 조회"""
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010100"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)
    try:
        start_price=int(res.json()['output']['stck_oprc']) ##시작가
        high_price=int(res.json()['output']['stck_hgpr'])   ##최고가
        low_price=int(res.json()['output']['stck_lwpr'])        ##최저가
        
        return start_price,high_price,low_price  ##시작가, 최고가, 최저가
    except:
        start_price,high_price,low_price=100000,100000,100000
        return start_price,high_price,low_price


def get_inquirePsblCancel():
    """취소가능 조회"""
    PATH = "uapi/domestic-stock/v1/trading/inquire-psbl-rvsecncl"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8036R",
        "custtype":"P",
    }
    params = {
        "ACNT_PRDT_CD": "01",
        "CANO": CANO,
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": "",
        "INQR_DVSN_1": "0",
        "INQR_DVSN_2": "0",
    }
    
    res = requests.get(URL, headers=headers, params=params)
    psblCancel_list = res.json()['output']
    sellPsblCancel_dic={}
    sellPsblCancel_list=[]
    buyPsblCancel_dic={}
    buyPsblCancel_list=[]
    # print(f"psblCancel_list:{psblCancel_list}")
    for psblCancel in psblCancel_list:
        if int(psblCancel['psbl_qty']) > 0 and psblCancel['sll_buy_dvsn_cd'] == '01' and psblCancel['pdno']==sym:
            sellPsblCancel_list.append(psblCancel['odno'])

        if int(psblCancel['psbl_qty']) > 0 and psblCancel['sll_buy_dvsn_cd'] == '02' and psblCancel['pdno']==sym:
            buyPsblCancel_list.append(psblCancel['odno'])
    sellPsblCancel_dic[sym] = sellPsblCancel_list
    buyPsblCancel_dic[sym] = buyPsblCancel_list

    # print(f"sellPsblCancel_list:{sellPsblCancel_list}")   
    # print(f"buyPsblCancel_list:{buyPsblCancel_list}") 
    return sellPsblCancel_dic,buyPsblCancel_dic  ##dict형식이며 리스트형의 원주문 번호가 추출된다


def cancel(odno="123456789"):
    """주식 주문취소"""  
    PATH = "uapi/domestic-stock/v1/trading/order-rvsecncl"
    URL = f"{URL_BASE}/{PATH}"
    data = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "KRX_FWDG_ORD_ORGNO": "91253",
        "ORGN_ODNO": odno,  # 주문번호
        "ORD_DVSN": "00",
        "RVSE_CNCL_DVSN_CD": "02", # 01:주문취소, 02:전량취소
        "ORD_QTY": "0",
        "ORD_UNPR":"0",
        "QTY_ALL_ORD_YN": "Y",
    }
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC0803U",
        "custtype":"P",
        "hashkey" : hashkey(data)
    }
    res = requests.post(URL, headers=headers, data=json.dumps(data))
    if res.json()['rt_cd'] == '0':
        # print(f"[취소성공]")
        return True
    else:
        print(f"[취소실패]")
        return False

def sell_cancel():
    sellPsblCancel_dic,buyPsblCancel_dic=get_inquirePsblCancel()
    if sym in sellPsblCancel_dic:
        for value in sellPsblCancel_dic[sym]:
            orgn_odno=value
            cancel(orgn_odno)
    else:
        print(f"▶매도취소주문이 없습니다.{sellPsblCancel_dic.items()}")
        time.sleep(0.1)

def sell_cancel_first(): #맨앞 취소)
    sellPsblCancel_dic,buyPsblCancel_dic=get_inquirePsblCancel()
    if sym in sellPsblCancel_dic and len(sellPsblCancel_dic[sym]) > 0:
        orgn_odno= sellPsblCancel_dic[sym][0]
        cancel(orgn_odno)
    else:
        # print(f"▶매도취소주문이 없습니다.{sellPsblCancel_dic.items()}")
        time.sleep(0.1)

def buy_cancel():
    sellPsblCancel_dic,buyPsblCancel_dic=get_inquirePsblCancel()
    if sym in buyPsblCancel_dic:
        for value in buyPsblCancel_dic[sym]:
            orgn_odno=value
            cancel(orgn_odno)
    else:
        print(f"▶매수취소주문이 없습니다.{buyPsblCancel_dic.items()}")
        time.sleep(0.1)



def buy(code="032350", qty="1",jj="jijung"):
    """주식 지정가 매수"""  
    PATH = "uapi/domestic-stock/v1/trading/order-cash"
    URL = f"{URL_BASE}/{PATH}"
    data = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "PDNO": code,
        "ORD_DVSN": "00",
        "ORD_QTY": str(int(qty)),
        "ORD_UNPR": str(int(jj)),
    }
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC0802U",
        "custtype":"P",
        "hashkey" : hashkey(data)
    }
    res = requests.post(URL, headers=headers, data=json.dumps(data))
    if res.json()['rt_cd'] == '0':
        # send_message(f"[매수 성공]{str(res.json())}")
        return True
    else:
        print(f"[매수 실패]{str(res.json())}")
        return False

def buy_marketPrice(code="114800", qty="1"):
    """주식 시장가 매수"""  
    PATH = "uapi/domestic-stock/v1/trading/order-cash"
    URL = f"{URL_BASE}/{PATH}"
    data = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "PDNO": code,
        "ORD_DVSN": "01",
        "ORD_QTY": str(int(qty)),
        "ORD_UNPR": "0",
    }
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC0802U",
        "custtype":"P",
        "hashkey" : hashkey(data)
    }
    res = requests.post(URL, headers=headers, data=json.dumps(data))
    if res.json()['rt_cd'] == '0':
        # send_message(f"[매수 성공]{str(res.json())}")
        return True
    else:
        print(f"[시장가매수 실패]{str(res.json())}")
        sys.exit()  # 프로그램 종료
    
def sell(code="032350", qty="1",jj="jijung"):
    """주식 지정가 매도"""
    PATH = "uapi/domestic-stock/v1/trading/order-cash"
    URL = f"{URL_BASE}/{PATH}"
    data = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "PDNO": code,
        "ORD_DVSN": "00",
        "ORD_QTY": str(int(qty)),
        "ORD_UNPR": str(int(jj)),
    }
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC0801U",
        "custtype":"P",
        "hashkey" : hashkey(data)
    }
    res = requests.post(URL, headers=headers, data=json.dumps(data))
    if res.json()['rt_cd'] == '0':
        # send_message(f"[매도 성공]{str(res.json())}")
        return True
    else:
        print(f"[매도 실패]{str(res.json())}")
        return False
def sell_marketPrice(code="114800", qty="1"):
    """주식 시장가 매도"""
    PATH = "uapi/domestic-stock/v1/trading/order-cash"
    URL = f"{URL_BASE}/{PATH}"
    data = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "PDNO": code,
        "ORD_DVSN": "01",
        "ORD_QTY": qty,
        "ORD_UNPR": "0",
    }
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC0801U",
        "custtype":"P",
        "hashkey" : hashkey(data)
    }
    res = requests.post(URL, headers=headers, data=json.dumps(data))
    if res.json()['rt_cd'] == '0':
        # send_message(f"[매도 성공]{str(res.json())}")
        return True
    else:
        send_message(f"[매도 실패]{str(res.json())}")
        return False
    


ACCESS_TOKEN = get_access_token()
engine = pyttsx3.init()
engine.say("접속")
engine.runAndWait()
print("🉑접속성공  ")   
print("-"*130) ;time.sleep(1)


In [ ]:
hamsu="▶응용 함수를 실행합니다."

def get_stock_name(): #보유하고 있는 주식만 가능한 주식명조회
    """주식명조회"""
    PATH = "uapi/domestic-stock/v1/trading/inquire-balance"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8434R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "AFHR_FLPR_YN": "N",
        "OFL_YN": "",
        "INQR_DVSN": "02",
        "UNPR_DVSN": "01",
        "FUND_STTL_ICLD_YN": "N",
        "FNCG_AMT_AUTO_RDPT_YN": "N",
        "PRCS_DVSN": "01",
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": ""
    }
    res = requests.get(URL, headers=headers, params=params)
    stock_list = res.json()['output1']
    stock_name = {}
    
    for stock in stock_list:
        if int(stock['hldg_qty']) > 0:
            stock_name[stock['pdno']] = stock['prdt_name']   ##주식한글명
    return stock_name

def get_stock_prdt_name(code='005930'): #주식명조회 all,어제종가,상장주식수,액면가
    """주식명조회"""
    PATH = "uapi/domestic-stock/v1/quotations/search-stock-info"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"CTPF1002R",
        "custtype":"P",
    }
    params = {
        "PDNO":code,
        "PRDT_TYPE_CD":"300"
    }  
    res = requests.get(URL, headers=headers, params=params)
    stock_dict = res.json()['output']
    stock_name = stock_dict['prdt_name'] #주식한글명
    stock_yesterday = int(stock_dict['bfdy_clpr']) #전일종가)
    stock_amount=int(stock_dict['lstg_stqt']) #상장주식수
    stock_value=int(stock_dict['papr'])   #액면가
    # for stock in stock_list:
    #     if int(stock['hldg_qty']) > 0:
    #         stock_name[stock['pdno']] = stock['prdt_name']   ##주식한글명
    return stock_name,stock_yesterday,stock_amount,stock_value

def get_period_trade_profit(code='122630'): #당일 실현손익만 조회
    """ 실현손익현황 조회"""
    from datetime import datetime
    today = datetime.now()
    formatted_date = today.strftime("%Y%m%d")
    PATH = "uapi/domestic-stock/v1/trading/inquire-period-profit"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8708R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": "01",
        "PDNO":code,
        "INQR_STRT_DT":formatted_date,
        "INQR_END_DT":formatted_date,
        "SORT_DVSN":"00",
        "INQR_DVSN":"00",
        "CBLC_DVSN":"00",
        "CTX_AREA_FK100":"",
        "CTX_AREA_NK100":""
        }
    res = requests.get(URL, headers=headers, params=params)
    # period_list=res.json()['output1']
    try:
        sellBuyPeriod_list = res.json()['output2']
        sillHyunProfit = int(sellBuyPeriod_list["tot_rlzt_pfls"])  # 실현손익
        return sillHyunProfit
    except:
        sillHyunProfit=1
        return sillHyunProfit
    
def get_periodProfit(code='122630',tday=5): #기간별 실현손익 조회(sym,days) 두개의 인자를 받는다.
    """ days단위로 기간 매매실현손익현황 조회"""
    from datetime import datetime, timedelta
    today = datetime.now()
    days_ago = today - timedelta(days=tday)
    formatted_date_ago = days_ago.strftime("%Y%m%d")
    formatted_today = today.strftime("%Y%m%d")
    
    PATH = "uapi/domestic-stock/v1/trading/inquire-period-profit"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8708R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": "01",
        "PDNO":code,
        "INQR_STRT_DT":formatted_date_ago,
        "INQR_END_DT":formatted_today,
        "SORT_DVSN":"00",
        "INQR_DVSN":"00",
        "CBLC_DVSN":"00",
        "CTX_AREA_FK100":"",
        "CTX_AREA_NK100":""
        }
    res = requests.get(URL, headers=headers, params=params)
    # period_list=res.json()['output1']
    try:
        sellBuyPeriod_list = res.json()['output2']
        periodProfit = int(sellBuyPeriod_list["tot_rlzt_pfls"])  # 실현손익
        return periodProfit
    except:
        periodProfit=1
        return periodProfit

def get_period_BuySell_AvgQtyAmount(code='122630',tday=5): # """ 기간별 매매평균,매매수량, 매매금액 합계 조회"""
    from datetime import datetime, timedelta
    today = datetime.now()
    days_ago = today - timedelta(days=tday)
    formatted_date_ago = days_ago.strftime("%Y%m%d")
    formatted_today = today.strftime("%Y%m%d")
    
    PATH = "uapi/domestic-stock/v1/trading/inquire-period-profit"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8708R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": "01",
        "PDNO":code,
        "INQR_STRT_DT":formatted_date_ago,
        "INQR_END_DT":formatted_today,
        "SORT_DVSN":"00",
        "INQR_DVSN":"00",
        "CBLC_DVSN":"00",
        "CTX_AREA_FK100":"",
        "CTX_AREA_NK100":""
        }
    res = requests.get(URL, headers=headers, params=params)
    try:
        sellBuyPeriod_list = res.json()['output2']
        time.sleep(0.1)
        periodBuyqtySum = int(sellBuyPeriod_list['buy_qty_smtl'])  # 매수수량
        periodSellqtySum = int(sellBuyPeriod_list['sll_qty_smtl'])  # 매도수량
        time.sleep(0.1)
        periodBuySumAmount = int(sellBuyPeriod_list['buy_tr_amt_smtl'])  # 매수금액
        periodSellSumAmount = int(sellBuyPeriod_list['sll_tr_amt_smtl'])  # 매도금액
        time.sleep(0.1)
        tot_fee=int(sellBuyPeriod_list['tot_fee'])  # 수수료
        tot_tltx=int(sellBuyPeriod_list['tot_tltx'])  # 세금
        totCost=tot_fee+tot_tltx
        return periodBuyqtySum,periodSellqtySum,periodBuySumAmount,periodSellSumAmount,totCost

    except:
        periodBuyqtySum,periodSellqtySum,periodBuySumAmount,periodSellSumAmount,totCost=1,1,1,1,1
        return periodBuyqtySum,periodSellqtySum,periodBuySumAmount,periodSellSumAmount,totCost


def get_today_BuySell_Qty(code='122630'): # """ 기간별 매매평균,매매수량, 매매금액 합계 조회"""
    from datetime import datetime, timedelta
    today = datetime.now()
    # days_ago = today - timedelta(days=tday)
    # formatted_date_ago = days_ago.strftime("%Y%m%d")
    formatted_today = today.strftime("%Y%m%d")
    
    PATH = "uapi/domestic-stock/v1/trading/inquire-period-profit"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8708R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": "01",
        "PDNO":code,
        "INQR_STRT_DT":formatted_today,
        "INQR_END_DT":formatted_today,
        "SORT_DVSN":"00",
        "INQR_DVSN":"00",
        "CBLC_DVSN":"00",
        "CTX_AREA_FK100":"",
        "CTX_AREA_NK100":""
        }
    res = requests.get(URL, headers=headers, params=params)
    try:
        sellBuyPeriod_list = res.json()['output2']
        todayBuyQ = int(sellBuyPeriod_list['buy_qty_smtl'])  # 매수수량
        todaySellQ = int(sellBuyPeriod_list['sll_qty_smtl'])  # 매도수량
        periodBuySumAmount = int(sellBuyPeriod_list['buy_tr_amt_smtl'])  # 매수금액
        periodSellSumAmount = int(sellBuyPeriod_list['sll_tr_amt_smtl'])  # 매도금액

        if todayBuyQ!=0 or todaySellQ!=0:
            if todayBuyQ==0:  todayBuyAvg=1
            else: todayBuyAvg=round(periodBuySumAmount/todayBuyQ)
            if todaySellQ==0: todaySellAvg=1
            else: todaySellAvg=round(periodSellSumAmount/todaySellQ)
            sAvg_sub_bAvg=todaySellAvg-todayBuyAvg
            tot_fee=int(sellBuyPeriod_list['tot_fee'])  # 수수료
            tot_tltx=int(sellBuyPeriod_list['tot_tltx'])  # 세금
            totCost=tot_fee+tot_tltx
            todayProfit=sAvg_sub_bAvg*todaySellQ-totCost   
        else:
            todayBuyAvg,todaySellAvg,todayProfit=1,1,1 

        return todayBuyQ,todaySellQ,todayBuyAvg,todaySellAvg,todayProfit
    
    except:
        todayBuyQ,todaySellQ,todayBuyAvg,todaySellAvg,todayProfit=1,1,1,1,1
        return todayBuyQ,todaySellQ,todayBuyAvg,todaySellAvg,todayProfit



def get_stock_balance_Qty_money(): # 당일 매수 매도 수량 조회
    PATH = "uapi/domestic-stock/v1/trading/inquire-balance"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8434R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "AFHR_FLPR_YN": "N",
        "OFL_YN": "",
        "INQR_DVSN": "02",
        "UNPR_DVSN": "01",
        "FUND_STTL_ICLD_YN": "N",
        "FNCG_AMT_AUTO_RDPT_YN": "N",
        "PRCS_DVSN": "01",
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": ""
    }
    res = requests.get(URL, headers=headers, params=params)
    try:
        stock_list = res.json()['output1']
        stock_todayBuyQ = {}
        stock_todaySellQ ={}
   
        for stock in stock_list:
            if int(stock['hldg_qty']) > 0:
                stock_todayBuyQ[stock['pdno']] = int(stock['thdt_buyqty'])  ##당일매수수량
                stock_todaySellQ[stock['pdno']] = int(stock['thdt_sll_qty'])   ##당일매도수량
            else:
                stock_todayBuyQ[stock['pdno']] = 0
                stock_todaySellQ[stock['pdno']] = 0
        return stock_todayBuyQ,stock_todaySellQ  ##stock_dic={'032350':'4500'}   즉 코드번호와 매입평균가격이 문자형 딕셔러니로 return된다.
    except:
        stock_todayBuyQ['122630']=1
        stock_todaySellQ['122630'] =1
        return stock_todayBuyQ,stock_todaySellQ

def todayBuySellQ(): #당일 매수 매도 수량 조회
    try:
        stock_todayBuyQ, stock_todaySellQ = get_stock_balance_Qty_money()
        todayBuyQ = stock_todayBuyQ.get(sym)
        todaySellQ = stock_todaySellQ.get(sym)
        return todayBuyQ, todaySellQ
    except:
        todayBuyQ,todaySellQ=1,1
        return todayBuyQ,todaySellQ
    
def get_todaySellBuyAvg(code="005930"): #당일 매도 매수 평균금액
    from datetime import datetime
    today = datetime.now()
    formatted_date = today.strftime("%Y%m%d")

    PATH = "uapi/domestic-stock/v1/trading/inquire-daily-ccld"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8001R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "INQR_STRT_DT": formatted_date,
        "INQR_END_DT": formatted_date,
        "SLL_BUY_DVSN_CD": "00",
        "INQR_DVSN": "00",
        "PDNO": code,
        "CCLD_DVSN": "01",  #체결
        "ORD_GNO_BRNO": "",
        "ODNO": "",
        "INQR_DVSN_3": "00",
        "INQR_DVSN_1": "",
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": ""
    }
    res = requests.get(URL, headers=headers, params=params)
    try:
        stock_list = res.json()['output1']
        todaySellAvg_list=[]
        todayBuyAvg_list=[]
        for stock in stock_list:
            if stock['sll_buy_dvsn_cd'] =="01": #sell
                todaySellAvg_list.append(int(stock['avg_prvs']))
            elif stock['sll_buy_dvsn_cd'] =="02":
                todayBuyAvg_list.append(int(stock['avg_prvs']))
        todaySellAvg = int(sum(todaySellAvg_list)/len(todaySellAvg_list)) if todaySellAvg_list else 0
        todayBuyAvg = int(sum(todayBuyAvg_list)/len(todayBuyAvg_list)) if todayBuyAvg_list else 0
        return todaySellAvg,todayBuyAvg ## 정수형으로 리턴을 받는다. 평가손익금액
    except:
        todaySellAvg,todayBuyAvg=1,1
        return todaySellAvg,todayBuyAvg

def get_stock_balance_evlu_pfls_amt(): # 당일 평가손익금액 조회
    PATH = "uapi/domestic-stock/v1/trading/inquire-balance"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8434R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "AFHR_FLPR_YN": "N",
        "OFL_YN": "",
        "INQR_DVSN": "02",
        "UNPR_DVSN": "01",
        "FUND_STTL_ICLD_YN": "N",
        "FNCG_AMT_AUTO_RDPT_YN": "N",
        "PRCS_DVSN": "01",
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": ""
    }
    res = requests.get(URL, headers=headers, params=params)
    try:
        stock_list = res.json()['output1']
        stock_evlu = {}
        for stock in stock_list:
            if int(stock['hldg_qty']) > 0:
                stock_evlu[stock['pdno']] = int(stock['evlu_pfls_amt'])   ##평가손익금액(evlu_pfls_am)
        return stock_evlu ## 정수형으로 리턴을 받는다. 평가손익금액
    except:
        stock_evlu['122630']=1
        return stock_evlu



def get_hldg_qtyPchs_avg_pricEvlu_amtEvlu_pfls_amt(): # 보유수량,매입평균금액,평가금액,평가손익금액
    PATH = "uapi/domestic-stock/v1/trading/inquire-balance"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"TTTC8434R",
        "custtype":"P",
    }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "AFHR_FLPR_YN": "N",
        "OFL_YN": "",
        "INQR_DVSN": "02",
        "UNPR_DVSN": "01",
        "FUND_STTL_ICLD_YN": "N",
        "FNCG_AMT_AUTO_RDPT_YN": "N",
        "PRCS_DVSN": "01",
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": ""
    }
    try:
        res = requests.get(URL, headers=headers, params=params)
        stock_list = res.json()['output1']
        hldg_qty = {}   # 보유수량
        pchs_avg_pric = {}  # 매입평균금액
        evlu_amt = {}  # 평가금액
        evlu_pfls_amt = {}  # 평가손익금액
        for stock in stock_list:
            if int(stock['hldg_qty']) > 0:
                hldg_qty[stock['pdno']] = int(stock['hldg_qty'])   ##보유수량(hldg_qty)
                pchs_avg_pric[stock['pdno']] = int(float(stock['pchs_avg_pric']))   ##매입평균가격(pchs_avg_pric)
                evlu_amt[stock['pdno']] = int(stock['evlu_amt']) #평가금액(evlu_amt)
                evlu_pfls_amt[stock['pdno']] = int(stock['evlu_pfls_amt'])   ##평가손익금액(evlu_pfls_am)
                
        return hldg_qty,pchs_avg_pric,evlu_amt,evlu_pfls_amt ## 보유수량,매입평균금액,평가금액,평가손익금액
    except:
        hldg_qty['122630']=1
        pchs_avg_pric['122630']=1
        evlu_amt['122630']=1
        evlu_pfls_amt['122630']=1
        return hldg_qty,pchs_avg_pric,evlu_amt,evlu_pfls_amt

def volumeRankMoney(): #거래량순위와 체결강도 100이상인 종목을 리스트로 리턴
    PATH = "uapi/domestic-stock/v1/quotations/volume-rank"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization": f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"FHPST01710000"}
    params = {
        "FID_COND_MRKT_DIV_CODE":"J",
        "FID_COND_SCR_DIV_CODE":"20171",
        "FID_INPUT_ISCD":"0000",
        "FID_DIV_CLS_CODE":"0",
        "FID_BLNG_CLS_CODE":"3",
        "FID_TRGT_CLS_CODE":"111111111",
        "FID_TRGT_EXLS_CLS_CODE":"000000",
        "FID_INPUT_PRICE_1":"0",
        "FID_INPUT_PRICE_2":"0",
        "FID_VOL_CNT":"0",
        "FID_INPUT_DATE_1":"0"
    }
    res = requests.get(URL, headers=headers, params=params)
    stock_list = res.json()['output']
    moneyRankName_dick={}
    originSymbol_list=[]
    symbol_list=[]
    for stock in stock_list:
        if int(stock['data_rank']) < 50 and (-10.0< float(stock['prdy_ctrt']) < 25.0) and (int(stock['stck_prpr']) < 100000):
            if not stock['hts_kor_isnm'].startswith("KODEX"):
                moneyRankName_dick[stock['mksc_shrn_iscd']] = stock['hts_kor_isnm']   #한글명
                originSymbol_list.append(stock['mksc_shrn_iscd'])
        
    for sym in originSymbol_list: 
        solidityBasic=get_tradeSolidity_symbol(sym)
        if solidityBasic > 100 :
            symbol_list.append(sym)
    return symbol_list,moneyRankName_dick


def get_tradeSolidity_symbol(code="114800"): #"""체결강도를 구하는 함수"""
    PATH = "uapi/domestic-stock/v1/quotations/inquire-ccnl"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization": f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"FHKST01010300"}
    params = {
    "fid_cond_mrkt_div_code": "J",
    "fid_input_iscd": code
    }
    res = requests.get(URL, headers=headers, params=params)
    try:
        stock_list = res.json()['output']
        solidi = float(stock_list[0]['tday_rltv'])   # 첫 번째 결과만 사용
    except KeyError:
        pass
    
    return solidi


def averageStockqtyOrderpossibleQty(): #평균가격, 보유수량, 주문가능수량을 정수로리턴
    try:
        stock_avg,stock_qty,ord_psbl_qty = get_stock_balance() 
        avg_price_num=int(float(stock_avg.get(sym)))
        stock_qty=int(stock_qty.get(sym))
        ord_psbl_qty=int(ord_psbl_qty.get(sym))
        return avg_price_num,stock_qty,ord_psbl_qty
    except:
        avg_price_num,stock_qty,ord_psbl_qty=1,1,1
        return avg_price_num,stock_qty,ord_psbl_qty
    
avg_price_num,stock_qty,ord_psbl_qty=averageStockqtyOrderpossibleQty()
engine = pyttsx3.init()
engine.say("ready!")
engine.runAndWait()

print(f"{sym}에 대한 함수가 준비되었습니다.")





In [ ]:
hoga_janryng="호가잔량 함수"
def get_hoga_janryng(code="005930"):  #호가잔량
    PATH = "uapi/domestic-stock/v1/quotations/inquire-asking-price-exp-ccn"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010200"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)
    sell_hoga_list=[]
    buy_hoga_list=[]
    sell_hoga_Q_list=[]
    buy_hoga_Q_list=[]
    try:
        # sellHogaJanryng = int(res.json()['output1']['total_askp_rsqn'])  # 매도호가잔량
        # buyHogaJanryng = int(res.json()['output1']['total_bidp_rsqn'])  # 매수호가잔량
        askp1=int(res.json()['output1']['askp1']) 
        askp2=int(res.json()['output1']['askp2']) 
        askp3=int(res.json()['output1']['askp3']) 
        askp4=int(res.json()['output1']['askp4']) 
        askp5=int(res.json()['output1']['askp5']) 
        askp6=int(res.json()['output1']['askp6']) 
        askp7=int(res.json()['output1']['askp7']) 
        askp8=int(res.json()['output1']['askp8']) 
        askp9=int(res.json()['output1']['askp9']) 
        askp10=int(res.json()['output1']['askp10']) 
        sell_hoga_list=[askp1,askp2,askp3,askp4,askp5,askp6,askp7,askp8,askp9,askp10]

        askp_rsqn1=int(res.json()['output1']['askp_rsqn1']) 
        askp_rsqn2=int(res.json()['output1']['askp_rsqn2']) 
        askp_rsqn3=int(res.json()['output1']['askp_rsqn3']) 
        askp_rsqn4=int(res.json()['output1']['askp_rsqn4']) 
        askp_rsqn5=int(res.json()['output1']['askp_rsqn5']) 
        askp_rsqn6=int(res.json()['output1']['askp_rsqn6']) 
        askp_rsqn7=int(res.json()['output1']['askp_rsqn7']) 
        askp_rsqn8=int(res.json()['output1']['askp_rsqn8']) 
        askp_rsqn9=int(res.json()['output1']['askp_rsqn9']) 
        askp_rsqn10=int(res.json()['output1']['askp_rsqn10']) 
        sell_hoga_Q_list=[askp_rsqn1,askp_rsqn2,askp_rsqn3,askp_rsqn4,askp_rsqn5,askp_rsqn6,askp_rsqn7,askp_rsqn8,askp_rsqn9,askp_rsqn10]


        bidp1=int(res.json()['output1']['bidp1']) 
        bidp2=int(res.json()['output1']['bidp2']) 
        bidp3=int(res.json()['output1']['bidp3']) 
        bidp4=int(res.json()['output1']['bidp4']) 
        bidp5=int(res.json()['output1']['bidp5']) 
        bidp6=int(res.json()['output1']['bidp6']) 
        bidp7=int(res.json()['output1']['bidp7']) 
        bidp8=int(res.json()['output1']['bidp8']) 
        bidp9=int(res.json()['output1']['bidp9']) 
        bidp10=int(res.json()['output1']['bidp10']) 
        buy_hoga_list=[bidp1,bidp2,bidp3,bidp4,bidp5,bidp6,bidp7,bidp8,bidp9,bidp10]

        bidp_rsqn1=int(res.json()['output1']['bidp_rsqn1']) 
        bidp_rsqn2=int(res.json()['output1']['bidp_rsqn2']) 
        bidp_rsqn3=int(res.json()['output1']['bidp_rsqn3']) 
        bidp_rsqn4=int(res.json()['output1']['bidp_rsqn4']) 
        bidp_rsqn5=int(res.json()['output1']['bidp_rsqn5']) 
        bidp_rsqn6=int(res.json()['output1']['bidp_rsqn6']) 
        bidp_rsqn7=int(res.json()['output1']['bidp_rsqn7']) 
        bidp_rsqn8=int(res.json()['output1']['bidp_rsqn8']) 
        bidp_rsqn9=int(res.json()['output1']['bidp_rsqn9']) 
        bidp_rsqn10=int(res.json()['output1']['bidp_rsqn10']) 
        buy_hoga_Q_list=[bidp_rsqn1,bidp_rsqn2,bidp_rsqn3,bidp_rsqn4,bidp_rsqn5,bidp_rsqn6,bidp_rsqn7,bidp_rsqn8,bidp_rsqn9,bidp_rsqn10]
    except KeyError:
        pass
    
    return sell_hoga_list,buy_hoga_list,sell_hoga_Q_list,buy_hoga_Q_list

def get_vs_yesterday(code="005930"):
    """전일대비 금액 및 상승하락률 조회"""
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010100"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)
    prdy_vrss=int(res.json()['output']['prdy_vrss'])
    prdy_ctrt=round(float((res.json()['output']['prdy_ctrt'])),2)
    
    return prdy_vrss,prdy_ctrt  #전일대비등락금액, 전일대비등락율
   
def get_investor(code="005930"):
    """투자자별 투자수량(개인,기관,외국인)"""
    PATH = "uapi/domestic-stock/v1/quotations/inquire-investor"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010900"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)
    prsn_ntby_qty=int(res.json()['output'][0]['prsn_ntby_qty']) #개인순매수수량
    orgn_ntby_qty=int(res.json()['output'][0]['orgn_ntby_qty']) #기관순매수수량
    frgn_ntby_qty=int(res.json()['output'][0]['frgn_ntby_qty']) #외국인순매수수량
    
    # 딕셔러리가 아닌 리스트에 리스트형식일 경우에는 [0]으로 첫번째 값임을 표시해야한다.
    return prsn_ntby_qty,orgn_ntby_qty,frgn_ntby_qty  

#어제종가를 구한다.
stock_name,stock_yesterday,stock_amount,stock_value=get_stock_prdt_name(sym)
print(f"{stock_name}  어제종가:{stock_yesterday:,}")

In [ ]:
play_start="게임시작"


import sys
from math import floor
import pygame
from random import randint
from pygame.locals import QUIT,MOUSEBUTTONDOWN

# current_price=get_current_price(sym)

WIDTH =19
HEIGHT= 15
SIZE = 50


pygame.init()
pygame.display.set_caption("WAtch")
SURFACE = pygame.display.set_mode((WIDTH*SIZE,HEIGHT*SIZE))
FPSCLOCK = pygame.time.Clock()
#######################################################


def main():

    smallfont=  pygame.font.SysFont(None,28)
    smallvsfont=  pygame.font.SysFont("malgungothic",20)
    largefont=  pygame.font.SysFont(None,72)
    stockfont = pygame.font.SysFont("malgungothic", 36)
    

    
    while True:
        sell_hoga_list,buy_hoga_list,sell_hoga_Q_list,buy_hoga_Q_list = get_hoga_janryng(sym)
        
        sell_Q=(sell_hoga_Q_list[0]/(sell_hoga_Q_list[0]+buy_hoga_Q_list[0]))*100
        buy_Q= 100-sell_Q
        col=(255,255,255)
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            
        # 그리기
        SURFACE.fill((100,100,100))
               
        # 선그리기
        for i in range(0,WIDTH*SIZE,SIZE):
            pygame.draw.line(SURFACE,(150,150,150),(i,0),(i,HEIGHT*SIZE))

        for j in range(0,HEIGHT*SIZE,SIZE):
            pygame.draw.line(SURFACE,(150,150,150),(0,j),(WIDTH*SIZE,j))

        #주식표시(삼성전자(005930) 현재가:     전일대비:   상승율)
        current_price=get_current_price(sym)
        stock_str= str(f"{stock_name}({sym})      {current_price:,}")
        if current_price > stock_yesterday:
            stock_str_image = stockfont.render(stock_str,True,(255,0,0))
        else:
            stock_str_image = stockfont.render(stock_str,True,(0,0,255))
        SURFACE.blit(stock_str_image,(10,10))

        prdy_vrss,prdy_ctrt= get_vs_yesterday(sym)
        stock_va_str= str(f"{prdy_vrss:,} ({prdy_ctrt})%")
        if prdy_vrss > 0:
            stock_vs_str_image = smallvsfont.render(stock_va_str,True,(255,0,0))
        else:
            stock_vs_str_image = smallvsfont.render(stock_va_str,True,(0,0,255))
        SURFACE.blit(stock_vs_str_image,(10,70))

        #호가잔량 직사각형 영역(전일대비 하락시 살짝파랑. 상승시 살짝 빨강)
        for i in range(1,6):
            shk=randint(-10,10)
            start_point=475-(sell_hoga_Q_list[i]//200//2)
            if sell_hoga_list[i] > stock_yesterday:
                col=(255-(i*20),0,0)
            elif sell_hoga_list[i] < stock_yesterday:
                col=(0,0,255-(i*20))
            pygame.draw.rect(SURFACE,col,(start_point+shk,(300-((i-1)*50)),sell_hoga_Q_list[i]//200,50))
        for i in range(1,6):
            shk=randint(-10,10)
            start_point=475-(buy_hoga_Q_list[i]//200//2)
            if buy_hoga_list[i] > stock_yesterday:
                col=(255-(i*20),0,0)
            elif buy_hoga_list[i] < stock_yesterday:
                col=(0,0,255-(i*20))
            pygame.draw.rect(SURFACE,col,(start_point+shk,(450+((i-1)*50)),buy_hoga_Q_list[i]//200,50))

        #호가 그리기  (전일대비 파랑/빨강)
        sell_hoga1_str= str(f"{sell_hoga_list[0]:,}").zfill(6) #전쟁터는 별도로하고
        sell_hoga1_image = smallfont.render(sell_hoga1_str,True,(255,255,255))
        SURFACE.blit(sell_hoga1_image,(350,370))
        for i in range(1,6):    #나머지 호가를 블릿한다
            sell_hoga1_str= str(f"{sell_hoga_list[i]:,}").zfill(6)
            sell_hoga1_image = smallfont.render(sell_hoga1_str,True,(255,255,255))
            SURFACE.blit(sell_hoga1_image,(250,320-((i-1)*50)))
        
        buy_hoga1_str= str(f"{buy_hoga_list[0]:,}").zfill(6)
        buy_hoga1_image = smallfont.render(buy_hoga1_str,True,(255,255,255))
        SURFACE.blit(buy_hoga1_image,(520,415))
        for i in range(1,6):    #나머지 호가를 블릿한다
            buy_hoga1_str= str(f"{buy_hoga_list[i]:,}").zfill(6)
            buy_hoga1_image = smallfont.render(buy_hoga1_str,True,(255,255,255))
            SURFACE.blit(buy_hoga1_image,(600,470+((i-1)*50)))

        
        #전쟁터 사각형
        shake=random.randint(-5,5)
        pygame.draw.ellipse(SURFACE,(250,0,0),(450,400-sell_Q+shake,50,sell_Q+shake))    
        pygame.draw.ellipse(SURFACE,(0,0,250),(450,400,50,buy_Q+shake)) 

        #개인,기관,외국인순매수수량표시
        prsn_ntby_qty,orgn_ntby_qty,frgn_ntby_qty =get_investor(sym)
        members=[prsn_ntby_qty,orgn_ntby_qty,frgn_ntby_qty]
        numbers=[]
        for mem in members: 
            numbers.append(abs(mem))
        total_length=950
        results = [round((n / sum((numbers))) * total_length, 2) for n in numbers]
        investor=['개인','기관','외인']
        
        distance=0
        for i in range(0,3):    #사각그래프영역표시
            shk=randint(-10,10)
            if members[i] > 0:
                col=(200,0,0) 
            else:
                col=(0,0,200)
            pygame.draw.rect(SURFACE,col,(distance,700,results[i],50))
            distance +=results[i]+5

        distance=0
        for i in range(0,3):    #숫자표시
            investor_str= str(f" {investor[i]} {members[i]:,}")
            investor_str_image = smallvsfont.render(investor_str,True,(255,255,255))
            if i ==0:
                SURFACE.blit(investor_str_image,(distance,705)) 
            elif i ==1:
                SURFACE.blit(investor_str_image,(distance,725))
            else:
                image_width = investor_str_image.get_width()
                SURFACE.blit(investor_str_image, (950 - image_width, 705))
            distance += results[i]  

        pygame.display.update()
        FPSCLOCK.tick(1)

if __name__ =='__main__':
    main()